# Midterm

I didn't use gradually decreasing step size because gamma will not converge.

In [6]:
import numpy as np

# Parameters
S, K, sigma, r, q, T = 49, 52, 0.27, 0.045, 0.02, 7 / 12.0
N = 10000_0000
dS, dSigma, dT, dr = 1e-8, 1e-8, 1e-8, 1e-8

# Helper function for option pricing
def option_price(S, sigma, r, T, z):
    S_T = S * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    C = np.exp(-r * T) * np.mean(np.maximum(S_T - K, 0))
    P = np.exp(-r * T) * np.mean(np.maximum(K - S_T, 0))
    return C, P

# Monte Carlo simulation
np.random.seed(42)
np.random.seed(42)
z = np.random.normal(0, 1, N)
C, P = option_price(S, sigma, r, T, z)

# Greeks calculation (except for Gamma)
def calc_greek(S, sigma, r, T, z, dParam, param_name):
    args = locals().copy()
    args[param_name] += dParam
    C_up, P_up = option_price(args['S'], args['sigma'], args['r'], args['T'], args['z'])
    args[param_name] -= 2 * dParam
    C_down, P_down = option_price(args['S'], args['sigma'], args['r'], args['T'], args['z'])
    return (C_up - C_down) / (2 * dParam), (P_up - P_down) / (2 * dParam)

# Gamma calculation
def calc_gamma(S, sigma, r, T, z, dS):
    C_up, P_up = option_price(S + dS, sigma, r, T, z)
    C_down, P_down = option_price(S - dS, sigma, r, T, z)
    return (C_up - 2 * C + C_down) / (dS ** 2), (P_up - 2 * P + P_down) / (dS ** 2)

# Calculate Greeks
Delta_C, Delta_P = calc_greek(S, sigma, r, T, z, dS, 'S')
Gamma_C, Gamma_P = calc_gamma(S, sigma, r, T, z, dS*200000) # need a larger dS for Gamma
Vega_C, Vega_P = calc_greek(S, sigma, r, T, z, dSigma, 'sigma')
Theta_C, Theta_P = calc_greek(S, sigma, r, T, z, dT, 'T')
Rho_C, Rho_P = calc_greek(S, sigma, r, T, z, dr, 'r')

# Display results
print(f"European Call Option Price: {C:.4f}, European Put Option Price: {P:.4f}")
print(f"Call Delta: {Delta_C:.4f}, Put Delta: {Delta_P:.4f}")
print(f"Call Gamma: {Gamma_C:.4f}, Put Gamma: {Gamma_P:.4f}")
print(f"Call Vega: {Vega_C:.4f}, Put Vega: {Vega_P:.4f}")
print(f"Call Theta: {Theta_C:.4f}, Put Theta: {Theta_P:.4f}")
print(f"Call Rho: {Rho_C:.4f}, Put Rho: {Rho_P:.4f}")


European Call Option Price: 3.0531, European Put Option Price: 5.2755
Call Delta: 0.4492, Put Delta: -0.5392
Call Gamma: 0.0385, Put Gamma: 0.0385
Call Vega: 14.6592, Put Vega: 14.6635
Call Theta: 3.8054, Put Theta: 2.4956
Call Rho: 11.0575, Put Rho: -18.4899


In [1]:
import numpy as np

# Given market price and other parameters
market_price = 6.36
S, K, r, q, T = 62, 66, 0.045, 0.015, 10 / 12.0
N = 10000_000

# Helper function for option pricing
def option_price(S, sigma, r, T, z):
    S_T = S * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    P = np.exp(-r * T) * np.mean(np.maximum(K - S_T, 0))
    return P

# Monte Carlo simulation
np.random.seed(42)
z = np.random.normal(0, 1, N)

# Bisection Method to find Implied Volatility
tol = 1e-5
max_iter = 100
sigma_low, sigma_high = 0.01, 2
for i in range(max_iter):
    sigma_guess = (sigma_low + sigma_high) / 2
    P_guess = option_price(S, sigma_guess, r, T, z)
    
    if abs(P_guess - market_price) < tol:
        break
    
    if P_guess > market_price:
        sigma_high = sigma_guess
    else:
        sigma_low = sigma_guess

implied_volatility = sigma_guess * 100  # Convert to percentage
print(f"Implied Volatility: {implied_volatility:.2f}%")


Implied Volatility: 22.50%
